# Tesla Smart Routing Reporter

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Description:

The Tesla Smart Routing Reporter is a Python-based intelligent system designed to analyze travel routes, traffic, and energy consumption for Tesla vehicles using a JSON dataset. It leverages OpenAI's GPT API integrated with the Toolhouse system design tools to process data, provide insights, and generate visually appealing reports. The app automates the evaluation of routes, calculates energy efficiency, and provides actionable information in markdown reports for electric vehicle optimization.

## Step 1. Boilerplate And Requirements Setup:

This initializes the environment and ensures all necessary dependencies are installed and environment variables are configured.

- `install_requirements()`: Installs the required dependencies using the requirements.txt file.

- `setup_env()`: Verifies the presence of critical environment variables (OPENAI_API_KEY and TOOLHOUSE_API_KEY).

- Key Libraries:

    - `dotenv`: Loads environment variables from a .env file.

    - `os`: Manages environment variables and system commands.

    - `IPython.display`: Clears output in a Jupyter notebook.

The setup ensures smooth operation by preparing the environment with required tools and configurations.

In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "TOOLHOUSE_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: API Integration and Toolhouse Setup: 

### 1. **Imports**
The script begins by importing essential libraries and tools:

- **OS Library:**  
  Used to access environment variables for securely storing and retrieving API keys.
  
- **OpenAI Client:**  
  Enables communication with OpenAI's GPT models for natural language processing.

- **Toolhouse Client:**  
  Facilitates the integration of advanced tools, such as a system design assistant.

- **Traceback Library:**  
  Captures detailed error messages for debugging purposes.

---

### 2. **API Key Retrieval**
API keys for **OpenAI** and **Toolhouse** are fetched from environment variables to securely authenticate requests. This method avoids hardcoding sensitive information directly in the script.

---

### 3. **Iteration Limit**
A constant is defined to limit the maximum number of iterations for tool-based interactions. This prevents infinite loops when handling complex tasks that may require multiple tool calls.

---

### 4. **Client Initialization**
- The **Toolhouse client** is initialized with the API key and linked to OpenAI as the provider, enabling it to use **Toolhouse tools** alongside GPT.
- The **OpenAI client** is also initialized to interact with GPT models and process user queries.

---

### 5. **Function Definition**
A wrapper function is defined to handle interactions with GPT models while incorporating **Toolhouse tools**.

---

### 6. **Parameters of the Function**
- **Prompt:**  
  The user’s input, describing the task or query to be processed.
  
- **Model:**  
  Specifies which GPT model to use (e.g., an optimized version of GPT-4).
  
- **System Role:**  
  Defines the assistant's behavior or persona (e.g., a helpful AI assistant).
  
- **Intermediate Output:**  
  If enabled, logs intermediate responses for debugging or monitoring progress.
  
- **Verbose Mode:**  
  Prints additional logs to aid in debugging.

---

### 7. **Message Initialization**
A message list is prepared to start the conversation, including:

- A **system role** defining the assistant's behavior.
- The **user’s prompt** describing their task.

---

### 8. **Initial API Call**
The script sends the initial request to OpenAI's GPT API with:

- The chosen **model**.
- The prepared **message list**.
- A list of **tools** (e.g., the system design assistant) provided by Toolhouse.

---

### 9. **Tool Execution**
The script evaluates the response from GPT to identify if tools are needed. If required:

- **Tools are executed**, and their outputs are incorporated into the conversation.
- This process repeats iteratively until all tool calls are resolved or the iteration limit is reached.

---

### 10. **Iterative Refinement**
The script repeatedly:

- Checks if **additional tool calls** are needed.
- Updates the **message list** with new responses or tool outputs.
- Refines the overall response with each iteration, ensuring accuracy and completeness.

---

### 11. **Error Handling**
If any issues arise during execution:

- The script captures **detailed error information** using the traceback library.
- This information is logged for debugging without crashing the application.

---

### 12. **Final Output**
Once all tools have been executed and the response is refined:

- The script returns the **final output**, which is a polished and complete answer to the user’s query.

In [24]:
## Example: LLM with Toolhouse
## Include the following tools in the Toolhouse bundle: thp-system-design-assistant
## Sendgrid, Web Scraper, Code Interpreter, and Memory tools are included in this bundle.

import os
from openai import OpenAI
from toolhouse import Toolhouse
import traceback

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TOOLHOUSE_API_KEY = os.getenv("TOOLHOUSE_API_KEY")
MAX_ITERS = 10
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider="openai")
openai = OpenAI(api_key=OPENAI_API_KEY)


def llm(
    prompt: str,
    model="gpt-4o-mini",
    system="You are a helpful AI assistant",
    intermediate_output=True,
    verbose=True,
) -> str:
    """Wrapper over Open AI LLM calls with Toolhouse Tools."""
    try:
        if verbose:
            print("System: ", system)
            print("Prompt: ", prompt)
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            tools=th.get_tools("thp-system-design-assistant"),
        )
        messages += th.run_tools(response)

        tool_calls = response.choices[0].message.tool_calls
        content = response.choices[0].message.content
        iters = 0

        while tool_calls and len(tool_calls) > 0 and iters < MAX_ITERS:
            print("Tools: ", tool_calls)
            if verbose or intermediate_output:
                if content:
                    print(f"Cycle {iters}: ", content)
                else:
                    print(f"Cycle {iters}: No content.")
            response = openai.chat.completions.create(
                model=model,
                messages=messages,
                tools=th.get_tools("thp-system-design-assistant"),
            )
            messages += th.run_tools(response)
            if verbose or intermediate_output:
                print(f"Generated intermediate response...")
            content = response.choices[0].message.content
            tool_calls = response.choices[0].message.tool_calls
            iters += 1

        if content and (verbose or intermediate_output):
            print(f"Final response: {content}")

        return content
    except Exception as e:
        traceback.print_exc()
        return "Sorry, failed to generate response.\nError: " + str(e)

## Step 3: Creating Routing Data

The **ROUTING_DATA Dictionary** provides a structured representation of various aspects needed for calculating routes, energy consumption, and comparing different travel options for the Tesla Model 3. Here's a breakdown of each key component:

### 1. **Locations**:  

This is a list of citie
s, each represented by:
- `id`: A unique identifier for each location (e.g., A, B, C).

- `coordinates`: The geographic coordinates (latitude and long
itude) of the city.

- `name`: The name of the city (e.g., San Francisco, Los Angeles, Fresno).



### 2. **Traffic Data**:  

This section defines the t
raffic conditions between pairs of locations:

- `from`: The starting location (using the `id`).

- `to`: The destination location (using the `id`).

- `distance_km`: The distance between the locations in kilometers.

- `traffic_level`: The traffic conditions (light, moderate, heavy).

- `speed_kmph`: The average speed of the route considering the traffic level.


### 3. **Elevation Data**:  

This defines the elevation (in meters) for each location, which is crucial for calculating energy consumption when traveling uphill or downhill.

### 4. **Vehicle Specifications**:  

This section provides details about the Tesla Model 3:

- `model`: The model of the vehicle.

- `efficiency_wh_per_km`: The energy efficiency of the vehicle, measured in watt-hours per kilometer (Wh/km).

- `battery_capacity_wh`: The total battery capacity of the vehicle in watt-hours (Wh).


---

### **Prompts List**:  

The **prompts list** consists of tasks the system needs to process using the **ROUTING_DATA**. Here's an overview of the prompts:

1. **Load and Display Data**:  

   - Load the routing data and display the locations, traffic, and elevation in an easy-to-read format.

2. **Create a Map with Routes**:  

   - Generate a map visualizing the routes between locations (A, B, and C), with distance and traffic labels.

3. **Calculate Travel Time**:  

   - Compute the travel time for each route by considering traffic and average speeds.

4. **Energy Usage Formula**:  

   - Write a formula to calculate additional energy used for elevation gains (for every 100 meters of elevation gain, the vehicle uses an additional 10 Wh/km).

5. **Energy-Efficient Route**:  

   - Compute the most energy-efficient route from San Francisco (A) to Los Angeles (B), considering distance, traffic, speed, and elevation.

6. **Comparison Table**:  

   - Generate a table comparing all routes between A and B, including metrics like travel time, elevation gain, energy consumption (Wh), and remaining battery percentage.

7. **Simulate Traffic and Elevation Changes**:  

   - Simulate how changes in traffic levels or elevation impact the energy consumption for the route from A to B.

8. **Bar Chart for Energy Consumption**:  

   - Create a bar chart to visualize energy consumption across the different routes between A and B.

9. **Heatmap for Reachable Locations**:  

   - Create a heatmap to show which locations can be reached from San Francisco (A) with 80% of the vehicle's battery, considering its energy specs.

---

### **Purpose of ROUTING_DATA**:  

The **ROUTING_DATA** dictionary provides all the necessary data points to evaluate different routes, factoring in distance, traffic, elevation, and energy consumption. It enables the system to generate detailed insights and perform complex calculations related to route selection, energy efficiency, and travel time for the Tesla Model 3.

### **Prompts Purpose**:  

The prompts direct the system to analyze the data and generate specific outputs like travel times, energy consumption, visual maps, and charts. These insights are useful for tasks such as optimizing routes for energy efficiency, understanding how traffic affects travel, and estimating how far the Tesla Model 3 can travel with different levels of battery charge.

In [25]:
## Tesla Smart Routing Reporter

ROUTING_DATA = {
    "locations": [
        {"id": "A", "coordinates": [37.7749, -122.4194], "name": "San Francisco"},
        {"id": "B", "coordinates": [34.0522, -118.2437], "name": "Los Angeles"},
        {"id": "C", "coordinates": [36.7783, -119.4179], "name": "Fresno"},
    ],
    "traffic_data": [
        {
            "from": "A",
            "to": "C",
            "distance_km": 300,
            "traffic_level": "moderate",
            "speed_kmph": 80,
        },
        {
            "from": "C",
            "to": "B",
            "distance_km": 200,
            "traffic_level": "heavy",
            "speed_kmph": 50,
        },
        {
            "from": "A",
            "to": "B",
            "distance_km": 500,
            "traffic_level": "light",
            "speed_kmph": 100,
        },
    ],
    "elevation_data": [
        {"id": "A", "elevation_m": 16},
        {"id": "B", "elevation_m": 71},
        {"id": "C", "elevation_m": 94},
    ],
    "vehicle_specs": {
        "model": "Tesla Model 3",
        "efficiency_wh_per_km": 150,
        "battery_capacity_wh": 75000,
    },
}


prompts = [
    "Load this JSON dataset and display the locations, traffic, and elevation data in a readable format.",
    "Create a map visualizing the routes between the locations A, B, and C with distances and traffic levels as labels.",
    "Calculate the travel time for each route considering traffic and average speeds provided in the dataset. Provide the result in a readable format.",
    "Write a formula to calculate additional energy usage for elevation gains, assuming that for every 100 meters of elevation gain, the vehicle uses an additional 10 Wh per km.",
    "Compute the most energy-efficient route from San Francisco (A) to Los Angeles (B) using distance, traffic, speed, and elevation data.",
    "Provide a table comparing all possible routes between A and B with metrics like travel time, elevation gain, energy consumption (Wh), and remaining battery percentage.",
    "Simulate how changes in traffic levels or elevation impacts the energy consumption for the A to B route.",
    "Plot a bar chart showing energy consumption for each route between A and B.",
    "Create a heatmap showing locations reachable from A with 80% battery given the vehicle's specs.",
]

## Step 4: File Existence and Section Check:

- **Imports**:
  
  - **`IPython.display` (Markdown, display):**  

    Imports for rendering markdown content in an IPython (Jupyter) notebook, useful for displaying formatted output.

  
  - **`json`:**  

    Imports the JSON module, though it is not used in this code snippet, it could be useful for handling JSON data in other parts of the script.
  
  - **`traceback`:**  

    This module is used for capturing and printing detailed error information, aiding in debugging.

- **Output File Path**:
  
  - **`output_file`:**  

    A variable defining the output file path (`test_output/tesla_routing_output.md`). This path can be modified as required for saving output data.

- **Function: `is_section_in_file(section_hint: str, file_path: str) -> bool`**:
  
  - **Purpose:**  

    Checks if a specified section (identified by `section_hint`) exists in the file located at `file_path`.
  
  - **How It Works:**  

    - The function attempts to open and read the content of the file (`file_path`).

    - If successful, it checks whether the `section_hint` is present in the file content.

    - If an error occurs (e.g., file not found or permission issues), it prints the error message and traceback details for debugging.

    - Returns `True` if the section is found, otherwise `False`.

- **Function: `create_file_if_not_exists_recursive(file_path: str)`**:
  
  - **Purpose:**  

    Creates the file at `file_path` if it doesn't already exist. This also ensures that the necessary parent directories are created if missing.
  
  - **How It Works:**

    - The function checks if the file at `file_path` exists.

    - If it doesn't exist, the function creates any necessary parent directories using `os.makedirs()` and then creates an empty file at `file_path`.

    - If an error occurs during this process (e.g., permission issues), the function captures and prints the error details, including the traceback.

- **Error Handling:**
  
  - Both functions use a `try-except` block to handle potential exceptions, ensuring that the script doesn't crash and provides useful debugging information in case of failure.

  - The `traceback.print_exc()` method prints the stack trace of the exception, providing detailed context about where and why an error occurred.

- **Summary:**
  
  - The code defines two utility functions:

    - `is_section_in_file`: Checks if a specific section exists in a file.

    - `create_file_if_not_exists_recursive`: Creates the file and necessary directories if they don't exist.
    
  - Both functions include error handling to ensure smooth execution and detailed debugging in case of failure.


In [26]:
from IPython.display import Markdown, display
import json
import traceback

output_file = "test_output/tesla_routing_output.md"  # change as required


def is_section_in_file(section_hint: str, file_path: str) -> bool:
    """Check if a section hint is present in a file."""
    try:
        with open(file_path, "r") as f:
            content = f.read()
        return section_hint in content
    except Exception as e:
        print("Error reading file:", str(e))
        traceback.print_exc()
        return False


def create_file_if_not_exists_recursive(file_path: str):
    """Create a file if it does not exist, creating parent directories if necessary."""
    try:
        if not os.path.exists(file_path):
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            with open(file_path, "w") as f:
                f.write("")
    except Exception as e:
        print("Error creating file:", str(e))
        traceback.print_exc()

## Step 5: Prompt Handling and Response Logging:

- ### create_file_if_not_exists_recursive(output_file):

    - This line ensures that the file (output_file), specified earlier as "test_output/tesla_routing_output.md", exists.  

    - If the file does not exist, it will be created.  

    - If necessary, parent directories will also be created.  

- ### for prompt in prompts::

    - This begins a loop that iterates over each item (prompt) in the prompts list.  

    - Each prompt in the list is a string that will be used to query the LLM (Language Learning Model).  

- ### if is_section_in_file(prompt, output_file)::

    - Checks if the section (identified by the current prompt) already exists in the output file (output_file).  

    - The function is_section_in_file will return True if the section is already present in the file and False otherwise.  

- ### print(f"Skipping prompt: {prompt}"):

    - If the section already exists in the output file, it prints a message saying that the prompt is being skipped.  

- ### continue:

    - This skips the rest of the code inside the loop for the current prompt and moves to the next prompt in the prompts list.  

- ### data_json = json.dumps(ROUTING_DATA):

    - Converts the ROUTING_DATA dictionary into a JSON-formatted string using the json.dumps() method.  

    - This string will be included in the prompt sent to the LLM.  

- ### prompt = f"{prompt}. Provide the response in a readable format.\n\n```json\n{data_json}\n```":

    - Modifies the current prompt by appending additional instructions and the JSON-formatted ROUTING_DATA string.  

    - The LLM is then asked to provide the response in a readable format based on the provided data.  

- ### print(f"Prompt: {prompt}"):

    - Prints the modified prompt (with added instructions and data) to the console.  
    - This allows you to see what the LLM is being asked to process.  

- ### response = llm(prompt=prompt):

    - Calls the llm() function (which interfaces with the OpenAI API and Toolhouse tools) to generate a response based on the modified prompt.  

    - The response from the LLM is stored in the response variable.  

- ### print("Response: ", response):

    - Prints the response generated by the LLM to the console, allowing you to see what the model output is.  

- ### if not is_section_in_file(prompt, output_file)::


    - Checks again if the section (based on the current prompt) already exists in the output file.  

    - If the prompt's section is not yet in the file, it proceeds to write the new response.  

- ### with open(output_file, "a") as f::

    - Opens the output file (output_file) in append mode ("a").  

    - This allows new content to be added to the end of the file without overwriting existing content.  

- ### f.write(f"## {prompt}\n" + response + "\n\n"):

    - Writes the prompt and its corresponding response to the output file.  

    - It prepends the prompt with ## to format it as a section header in Markdown.  

    - The response is written on the next line, followed by two newline characters for proper spacing.  

- ### print("\n\n"):

    - Prints two newline characters to create some space in the console output for clarity.  

- ### print(f"🚀 All prompts completed. Check the output file ('{output_file}') for the responses."):

    - After processing all the prompts, this message is printed to inform the user that all prompts have been processed,  
    and they can now check the output file for the detailed responses.  

### **Summary of Code Workflow**:

The code first ensures that the output file exists.  
Then, for each prompt in the list prompts, it checks if the section has already been processed (and saved in the output file).  
If the section exists, it skips the prompt; otherwise, it prepares the prompt and sends it to the LLM for processing.  
The generated response is then written to the output file, formatted in Markdown.  
Once all prompts are processed, the code informs the user that the task is complete and to check the output file for the results.  
This approach prevents the code from duplicating content in the output file and ensures that only new prompts are processed and saved.  


In [ ]:
create_file_if_not_exists_recursive(output_file)

for prompt in prompts:
    if is_section_in_file(prompt, output_file):
        print(f"Skipping prompt: {prompt}")
        continue
    data_json = json.dumps(ROUTING_DATA)
    prompt = f"{prompt}. Provide the response in a readable format.\n\n```json\n{data_json}\n```"
    print(f"Prompt: {prompt}")
    response = llm(prompt=prompt)
    print("Response: ", response)
    if not is_section_in_file(prompt, output_file):
        with open(output_file, "a") as f:
            f.write(f"## {prompt}\n" + response + "\n\n")
    print("\n\n")

print(
    f"🚀 All prompts completed. Check the output file ('{output_file}') for the responses."
)

## Conclusion:

The Tesla Smart Routing Reporter is a powerful tool for optimizing Tesla Model 3 routes by analyzing traffic, elevation, and energy consumption data. It efficiently processes JSON datasets, calculates travel times, energy efficiency, and provides actionable insights. By leveraging OpenAI's GPT models and Toolhouse, the app automates the generation of detailed reports, visualizations, and comparisons, ultimately helping drivers make smarter decisions for energy-efficient routes and better vehicle performance.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>
